# Digging deeper into "simple_cnn" function in fast.ai.layers

In [1]:
from fastai.vision import *
from fastai.layers import *
from fastai.callbacks import *

There's a function in fastai.layers called "simple_cnn"

In [2]:
def simple_cnn(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None, bn=False) -> nn.Sequential:
    "CNN with `conv_layer` defined by `actns`, `kernel_szs` and `strides`, plus batchnorm if `bn`."
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i],
              norm_type=(NormType.Batch if bn and i<(len(strides)-1) else None)) for i in range_of(strides)]
    layers.append(PoolFlatten())
    return nn.Sequential(*layers)

How does this function work?

Let's build the model first and dig deeper to it!

In [3]:
model = simple_cnn((3,16,16,2))

## The first three lines

In [ ]:
# in the above example
nl = len(actns)-1 #number of layers = 3
kernel_szs = ifnone(kernel_szs, [3]*nl) #kernel_sizes = [3,3,3]
strides    = ifnone(strides   , [2]*nl) #strides = [2,2,2]

I got stuck on this part the most... essentially the 3dimensional aspect is very important

In [5]:
model.parameters

<bound method Module.parameters of Sequential(
  (0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (1): Sequential(
    (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (2): Sequential(
    (0): Conv2d(16, 2, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (3): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten()
  )
)>

In [11]:
print(model_summary(model))

Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [1, 16, 32, 32]      448        True      
______________________________________________________________________
ReLU                 [1, 16, 32, 32]      0          False     
______________________________________________________________________
Conv2d               [1, 16, 16, 16]      2,320      True      
______________________________________________________________________
ReLU                 [1, 16, 16, 16]      0          False     
______________________________________________________________________
Conv2d               [1, 2, 8, 8]         290        True      
______________________________________________________________________
ReLU                 [1, 2, 8, 8]         0          False     
______________________________________________________________________
AdaptiveAvgPool2d    [1, 2, 1, 1]         0          False     
______________________________________________________________

In [24]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

0.0.weight tensor([[[[ 3.9805e-01, -1.5744e-02, -8.3080e-02],
          [ 3.8779e-01,  6.1102e-01,  2.7343e-01],
          [ 2.9187e-01,  2.0947e-01,  2.4502e-01]],

         [[ 4.0543e-01, -7.4264e-02, -2.6748e-01],
          [ 1.3807e-01,  6.0930e-01, -2.0764e-01],
          [ 1.5921e-01,  2.0896e-01, -1.6753e-01]],

         [[-1.8564e-01,  1.2214e-01, -7.4731e-03],
          [ 3.5292e-01, -1.4793e-01,  1.1959e-01],
          [ 2.0565e-01,  3.0185e-01, -1.9265e-02]]],


        [[[-9.1971e-02, -3.8969e-02,  7.8169e-02],
          [ 1.3087e-01,  1.2898e-01,  2.2260e-01],
          [-1.4655e-01,  3.3976e-01, -1.1281e-01]],

         [[-1.6177e-02,  2.3368e-01,  4.6686e-01],
          [-2.7250e-01, -3.3415e-01,  3.0356e-01],
          [-7.0779e-02, -4.7289e-01, -1.2277e-01]],

         [[ 5.9208e-01,  3.1741e-01,  2.2373e-01],
          [ 1.5093e-01, -6.1338e-02, -3.2911e-02],
          [-1.5035e-01, -1.7318e-01, -3.6767e-01]]],


        [[[-8.4889e-02,  2.2199e-01, -2.5705e-01],
    

## conv_layer part

So what is `conv_layer` in this part of the code?

In [ ]:
layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i],
              norm_type=(NormType.Batch if bn and i<(len(strides)-1) else None)) for i in range_of(strides)]

In [ ]:
def conv_layer(ni:int, nf:int, ks:int=3, stride:int=1, padding:int=None, bias:bool=None, is_1d:bool=False,
               norm_type:Optional[NormType]=NormType.Batch,  use_activ:bool=True, leaky:float=None,
               transpose:bool=False, init:Callable=nn.init.kaiming_normal_, self_attention:bool=False):
    "Create a sequence of convolutional (`ni` to `nf`), ReLU (if `use_activ`) and batchnorm (if `bn`) layers."
    if padding is None: padding = (ks-1)//2 if not transpose else 0
    bn = norm_type in (NormType.Batch, NormType.BatchZero)
    if bias is None: bias = not bn
    conv_func = nn.ConvTranspose2d if transpose else nn.Conv1d if is_1d else nn.Conv2d
    conv = init_default(conv_func(ni, nf, kernel_size=ks, bias=bias, stride=stride, padding=padding), init)
    if   norm_type==NormType.Weight:   conv = weight_norm(conv)
    elif norm_type==NormType.Spectral: conv = spectral_norm(conv)
    layers = [conv]
    if use_activ: layers.append(relu(True, leaky=leaky))
    if bn: layers.append((nn.BatchNorm1d if is_1d else nn.BatchNorm2d)(nf))
    if self_attention: layers.append(SelfAttention(nf))
    return nn.Sequential(*layers)

Basically, conv2d with relu without tuning any parameters with kaiming_normal initalization

## PoolFlatten()

Finally, PoolFlatten adds a nn.AdaptiveApgPool2d 

In [ ]:
def PoolFlatten()->nn.Sequential:
    "Apply `nn.AdaptiveAvgPool2d` to `x` and then flatten the result."
    return nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten())